In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../../read")

from read.utils.table import linearize

In [3]:
from sentence_transformers import SentenceTransformer, util
import os
import jaro

sentence_selector = SentenceTransformer("../temp/sentence_selection/bart-triplet-2")

def check_select(sent1, sent2):
    embeddings1 = sentence_selector.encode(sent1, convert_to_tensor=True)
    embeddings2 = sentence_selector.encode(sent2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)[0][0].item()
    return cosine_scores

/nas/home/minhpham/miniconda3/envs/read/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from kilt.knowledge_source import KnowledgeSource
ks = KnowledgeSource()


import pandas as pd

df = pd.read_json("../data/totto/augmented/dev.jsonl", lines=True)

In [12]:
from blingfire import text_to_sentences

golds = []
indices = []
preds = []

k2metrics = {k: [RetrievalHitRate(k=k), RetrievalRPrecision(k=k), RetrievalRecall(k=k), RetrievalPrecision(k=k), RetrievalMRR(k=k), RetrievalMAP(k=k)] for k in [1, 5, 10]}


for idx, row in df.iterrows():
    linearized_table = linearize(row["table"])
    page = ks.get_page_by_title(row["table_page_title"])
    
    for passage in page["text"]:
        for sent in text_to_sentences(passage).split("\n"):
            gold = jaro.jaro_winkler_metric(sent, row["sentence"]) > 0.9
            indices.append(idx)
            golds.append(gold)
            preds.append(check_select(linearized_table, sent))
            # print(sent, row["sentence"], jaro.jaro_winkler_metric(sent, row["sentence"]), preds[-1])
            
    if (idx + 1) % 10 == 0:
        print(idx)

        preds_tensor = torch.tensor(preds)
        golds_tensor = torch.tensor(golds)
        indices_tensor = torch.tensor(indices)

        for k, metrics in k2metrics.items():
            for metric in metrics:
                result[f"{metric.__class__.__name__}@{k}"] = metric(preds_tensor, golds_tensor, indices_tensor)
        print(result)


9
{'RetrievalHitRate@1': tensor(0.6000), 'RetrievalRPrecision@1': tensor(0.6000), 'RetrievalRecall@1': tensor(0.6000), 'RetrievalPrecision@1': tensor(0.6000), 'RetrievalMRR@1': tensor(0.6441), 'RetrievalMAP@1': tensor(0.6441), 'RetrievalHitRate@5': tensor(0.7000), 'RetrievalRPrecision@5': tensor(0.6000), 'RetrievalRecall@5': tensor(0.7000), 'RetrievalPrecision@5': tensor(0.1400), 'RetrievalMRR@5': tensor(0.6441), 'RetrievalMAP@5': tensor(0.6441), 'RetrievalHitRate@10': tensor(0.8000), 'RetrievalRPrecision@10': tensor(0.6000), 'RetrievalRecall@10': tensor(0.8000), 'RetrievalPrecision@10': tensor(0.0800), 'RetrievalMRR@10': tensor(0.6441), 'RetrievalMAP@10': tensor(0.6441)}
19
{'RetrievalHitRate@1': tensor(0.3500), 'RetrievalRPrecision@1': tensor(0.3500), 'RetrievalRecall@1': tensor(0.3500), 'RetrievalPrecision@1': tensor(0.3500), 'RetrievalMRR@1': tensor(0.4392), 'RetrievalMAP@1': tensor(0.4392), 'RetrievalHitRate@5': tensor(0.5500), 'RetrievalRPrecision@5': tensor(0.3500), 'RetrievalRe

TypeError: 'NoneType' object is not subscriptable

In [7]:
from read.utils.table import linearize
from torchmetrics import RetrievalHitRate, RetrievalRPrecision, RetrievalRecall, RetrievalPrecision, RetrievalMRR, RetrievalMAP


k2metrics = {k: [RetrievalHitRate(k=k), RetrievalRPrecision(k=k), RetrievalRecall(k=k), RetrievalPrecision(k=k), RetrievalMRR(k=k), RetrievalMAP(k=k)] for k in [1, 5, 10]}

import torch

result = {}

preds = torch.tensor(preds)
golds = torch.tensor(golds)
indices = torch.tensor(indices)

for k, metrics in k2metrics.items():
    for metric in metrics:
        result[f"{metric.__class__.__name__}@{k}"] = metric(preds, golds, indices)

In [8]:
result

{'RetrievalHitRate@1': tensor(0.4200),
 'RetrievalRPrecision@1': tensor(0.4200),
 'RetrievalRecall@1': tensor(0.4100),
 'RetrievalPrecision@1': tensor(0.4200),
 'RetrievalMRR@1': tensor(0.5083),
 'RetrievalMAP@1': tensor(0.5083),
 'RetrievalHitRate@5': tensor(0.6200),
 'RetrievalRPrecision@5': tensor(0.4200),
 'RetrievalRecall@5': tensor(0.6200),
 'RetrievalPrecision@5': tensor(0.1280),
 'RetrievalMRR@5': tensor(0.5083),
 'RetrievalMAP@5': tensor(0.5083),
 'RetrievalHitRate@10': tensor(0.7000),
 'RetrievalRPrecision@10': tensor(0.4200),
 'RetrievalRecall@10': tensor(0.7000),
 'RetrievalPrecision@10': tensor(0.0720),
 'RetrievalMRR@10': tensor(0.5083),
 'RetrievalMAP@10': tensor(0.5083)}

In [4]:
from pymongo import MongoClient
 
 
Client = MongoClient()
myclient = MongoClient('localhost', 27017)

In [22]:
myclient["kilt"].list_collection_names({})

ValueError: 'session' argument must be a ClientSession or None.

In [23]:
myclient["kilt"]["knowledgesource"].count_documents({})

5903530